In [1]:
using Interact
using RigidBodyDynamics
using DataFrames
using SimpleGradientDescent
using Ipopt

In [2]:
import Flash
reload("Flash")

adding: /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/DrakeVisualizer/src/lcmtypes to the python path


In [3]:
squished = Flash.DepthData.read_point_cloud(open("data/squishable_squished_xyzrgb.txt"))

PointCloud with origin: [1.37689,0.775602,1.4848] containing 25164 points

In [4]:
Flash.DepthData.render_lcmgl(squished)

In [5]:
unsquished = Flash.DepthData.read_point_cloud(open("data/squishable_unsquished_xyzrgb.txt"))

PointCloud with origin: [1.38246,0.768824,1.48581] containing 25571 points

In [6]:
Flash.DepthData.render_lcmgl(unsquished)

In [7]:
model = Flash.Models.squishable()
state = Flash.ManipulatorState(model)
Flash.draw(state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 294xFixedSizeArrays.Point{3,Float64},     faces: 584xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f6444227510>))

In [8]:
point_cloud = unsquished
Flash.DepthData.render_lcmgl(point_cloud)
sensed_points = [p.position for p in point_cloud.points];

In [9]:
function test_gradient_descent(model, sensed_points, loops, solver)
    for i = 1:loops
        
        function callback{T}(x::AbstractVector{T}, c)
            x_value = Flash.value.(x)
            state = Flash.ManipulatorState(model)
            Flash.GradientDescent.unflatten!(state, x_value)
            Flash.draw(state, false)
        end
        
        x_estimated = Flash.flatten(Flash.ManipulatorState(model))
        xstar = Flash.Tracking.estimate_state(model, sensed_points, x_estimated; callback=callback, solver=solver)
        state = Flash.ManipulatorState(model)
        Flash.GradientDescent.unflatten!(state, xstar)
        Flash.draw(state, true)
    end
end


test_gradient_descent (generic function with 1 method)

In [10]:
num_vars = Flash.num_states(model)
solver = NaiveSolver(num_vars; rate=0.01, max_step=0.1, iteration_limit=50, 
        precondition_divisors=vcat([1 for i in 1:7], [1e-3 for i in 8:num_vars]))
test_gradient_descent(model, sensed_points, 1, solver);

In [11]:
@time test_gradient_descent(model, sensed_points, 1, solver);

  5.354626 seconds (45.01 M allocations: 3.756 GB, 7.51% gc time)


In [12]:
ipopt_solver = IpoptSolver(print_level=1, max_iter=50)
test_gradient_descent(model, sensed_points, 1, ipopt_solver);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

